# 카카오 비전 API

카카오에서 최근에 흥미로운 서비스를 시작했습니다. 컴퓨터 비전(Computer Vision) API로 이미지 분석을 해주는 것이지요. 카카오 측에서는 아래와 같이 설명하고 있습니다.

> 이미지를 분석해서 콘텐츠 형태와 의미를 이해하고, 이미지를 활용할 수 있게 도와주는 API입니다.

현재 제공되는 API는 다음과 같습니다. 

- 얼굴 검출
- 상품 검출
- 썸네일 생성
- 성인 이미지 판별

특별히 새로운것은 아니지만, 카카오에서 공개한 API의 성능을 확인하고자 사용해봤습니다. 자세한 내용은 [사이트](https://developers.kakao.com/docs/restapi/vision)를 참고하세요.

# API 준수사항

API 사용시 다음과 같은 사항을 준수해야 합니다.

- 서버에서 호출하는 경우 REST API 키를 사용해 호출합니다.
- 모든 API는 file 또는 image_url 중 하나의 값을 지정해야 합니다.
- file에 업로드되는 이미지와 image_url에 지정되는 이미지는 png 또는 jpg 포맷만 지원합니다.
- file을 업로드하는 경우는 "multipart/form-data"로 POST 되어야 합니다.
- image_url로 호출 하는 경우는 "application/x-www-form-urlencoded"로 POST 되어야 합니다


# 활용 예제 살펴보기

카카오에서 간단한 활용 `파이썬` 코드를 제공하고 있습니다. 거기서 **얼굴을 추출해 모자이크 처리하는 예제**가 흥미로워 보였습니다. 카카오 API로 얼굴의 좌표를 인식하고 `pillow` 라이브러리를 이용해서 잘라내고 리사이즈를 해서 다시 붙여(모자이크처리) 넣습니다.  

예제로 사용할 사진은 아래와 같이 준비했습니다.

1. 원숭이 사진
![monkey](https://i.imgur.com/HIp4WAQ.jpg)
2. 단체사진
![peoples](https://i.imgur.com/9n0fVoI.jpg)

사용한 코드는 아래와 같습니다. 
- 본인의 API 키를 넣어야 작동합니다.
- 변환할 사진 파일은 jpg 혹은 png 포멧이어야 합니다.
- 사진 파일의 폴더위치를 말해주거나, 기본값으로 같은 폴더에 있으면 작동하게 만들었습니다.
- 모자이크 처리된 파일은 파일명에 *mosaic_*를 붙여서 생성됩니다.

In [1]:
import sys
import argparse
import requests
from PIL import Image, ImageFilter
import glob
import os

API_URL = 'https://kapi.kakao.com/v1/vision/face/detect'
# My Kakao api key, 본인 계정의 API 키로 변경해서 사용하세요.
MYAPP_KEY = 'b5f1dde7995c36168408b69308ef2186'

def detect_face(filename):
    headers = {'Authorization': 'KakaoAK {}'.format(MYAPP_KEY)}

    try:
        files = { 'file' : open(filename, 'rb')}
        resp = requests.post(API_URL, headers=headers, files=files)
        resp.raise_for_status()
        return resp.json()
    except Exception as e:
        print(str(e))
        sys.exit(0)

def mosaic(filename, detection_result):
    image = Image.open(filename)

    for face in detection_result['result']['faces']:
        x = int(face['x']*image.width)
        w = int(face['w']*image.width)
        y = int(face['y']*image.height)
        h = int(face['h']*image.height)
        box = image.crop((x,y,x+w, y+h))
        # 모자이크 강도를 조절하려면 사이즈를 조절하세요.
        box = box.resize((20,20), Image.NEAREST).resize((w,h), Image.NEAREST)
        image.paste(box, (x,y,x+w, y+h))
    return image



if __name__ == "__main__":
    parser = argparse.ArgumentParser(description='Mosaic all of faces.')
    parser.add_argument('-f', '--folder', type=str, nargs='?', default="./.",
                        help='image file to hide faces', required = True)
    args = parser.parse_args()
    # Make files lists
    files = glob.glob('*.jpg') + glob.glob('*.jpeg') + glob.glob('*.png')
    # make loop
    for i in files:
        head, tail = os.path.split(i)
        try:
            detection_result = detect_face(i)
            image = mosaic(i, detection_result)           
            image.save(head+'mosaic_'+tail,'JPEG')
            print('! Add mosaic on : ' + tail)
        except:
            print('! Face does not detected on : ' + tail) 
    
    print('!!  All process finished  !!')


!!  All process finished  !!


# 결과

1. 원숭이 사진
![](https://i.imgur.com/SMNZSM4.jpg)
2. 단체사진
![](https://i.imgur.com/DwBjyNp.jpg)

얼굴을 인식해서 모자이크 처리가 되는것을 볼수 있습니다. 이처럼 카카오에서 제공하는 API는 꽤나 강력합니다. 다만 얼굴 인식외에 상품 검출과 성인이미지 검출은 아직 인식률이 부족한것 같아서 앞으로 업데이트를 기대합니다.
